In [34]:
import pandas as pd
import sqlalchemy as sql
import pyads
import pymssql
import pypyodbc as odbc
from sklearn import preprocessing
from sklearn.decomposition import PCA
import seaborn as sns
from statsmodels.formula.api import ols

In [35]:
#plc = pyads.Connection('5.119.159.52.1.1', pyads.PORT_TC3PLC1)

#plc.open()

In [36]:
#plc.read_by_name('GVL.EL3443.MDP5001_341_Input.MDP5001_341_L1_Basic_Voltage')

In [37]:
#plc.close()

In [38]:
Driver = 'SQL SERVER'
Server = 'AYO-NB8'
Database = 'PyADS'

connection_string = f"""
    DRIVER={{{Driver}}};
    SERVER={Server};
    Database={Database};
    Trust_Connection=yes
"""
conn = odbc.connect(connection_string)

In [39]:
#query = "INSERT INTO EL3443 (id, fVoltage, fCurrent, fAcPower, fApPower, fRePower, fPWF, fFreq) VALUES (?, ?, ?, ?, ?, ?, ?, ?)"
#val = (4, 1, 1, 1, 1, 1, 1, 1)

#cursor.execute(query, val)
#conn.commit()

In [40]:
cursor = conn.cursor()
new_query = "SELECT * FROM EL3443"
cursor.execute(new_query)
#result = cursor.fetchall()
ELdata = pd.DataFrame(cursor, columns=[i[0] for i in cursor.description])

In [41]:
conn.close()

In [42]:
ELdata = ELdata.set_index('id')
#data = data.round(5)
ELdata.head()

,fvoltage,fcurrent,facpower,fappower,frepower,fpqf,ffreq
id,,,,,,,
1,242.704193,-0.008795,-0.027324,2.134490,2.134315,0.996691,50.031918
2,242.587845,-0.008803,-0.051194,2.135606,2.134993,0.996662,50.030983
3,242.713181,-0.008814,-0.045052,2.139300,2.138825,0.996662,50.028816
4,242.699081,-0.008842,-0.049646,2.145965,2.145391,0.996659,50.027889
5,242.760391,-0.008795,-0.064997,2.134982,2.133992,0.996655,50.025177


In [43]:
ELdata.describe()

,fvoltage,fcurrent,facpower,fappower,frepower,fpqf,ffreq
count,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000,394.000000
mean,242.280475,-0.008857,-0.045259,2.145936,2.145413,0.996646,49.968373
std,0.424530,0.000089,0.014155,0.019536,0.019432,0.000107,0.018630
min,241.151184,-0.009130,-0.087093,2.092818,2.092575,0.996406,49.930252
25%,241.995247,-0.008921,-0.054708,2.131438,2.131093,0.996548,49.958230
50%,242.357666,-0.008853,-0.045096,2.146131,2.145712,0.996656,49.966391
75%,242.629974,-0.008793,-0.034587,2.159699,2.159124,0.996727,49.977542
max,243.039902,-0.008628,-0.009690,2.203494,2.203084,0.996811,50.031918


In [44]:
#data['fvoltage'].plot(kind='kde')

In [45]:
#preprocessing.normalize(data.iloc[:,[0,1,2,3,4,5,6]])

In [46]:
pca = PCA()
pca.fit(ELdata)
pca.explained_variance_ratio_

array([9.94320769e-01, 3.32056312e-03, 1.48397960e-03, 8.74654421e-04,
       2.51272492e-08, 9.17089483e-09, 1.13195151e-13])

In [47]:
pd.DataFrame(pca.components_, columns=ELdata.columns)

,fvoltage,fcurrent,facpower,fappower,frepower,fpqf,ffreq
0,-0.999234,-0.000130,-0.013210,0.022722,0.022436,-0.000185,-1.835560e-02
1,0.028454,-0.002810,-0.149120,0.681137,0.678146,-0.000962,2.304657e-01
2,-0.024811,0.000696,-0.048577,-0.168614,-0.169737,-0.000782,9.694237e-01
3,0.010292,0.000348,-0.987425,-0.084255,-0.104981,-0.000722,-8.225216e-02
4,0.000170,-0.000444,0.002205,0.062061,-0.062959,-0.996082,-9.180677e-04
5,-0.000005,-0.003489,0.014692,0.704368,-0.704144,0.088427,3.274186e-05
6,0.000037,-0.999990,-0.000009,-0.004549,0.000421,0.000136,9.434906e-07


In [48]:
#sns.pairplot(data, vars=['fvoltage', 'fcurrent', 'facpower', 'fappower', 'frepower', 'fpqf', 'ffreq'])

In [49]:
ELdata.corr() #corrosion matrix

,fvoltage,fcurrent,facpower,fappower,frepower,fpqf,ffreq
fvoltage,1.000000,0.619057,0.395878,-0.492615,-0.489024,0.732461,0.418261
fcurrent,0.619057,1.000000,0.395410,-0.988401,-0.987677,0.605058,0.131612
facpower,0.395878,0.395410,1.000000,-0.361606,-0.348223,0.428582,0.088216
fappower,-0.492615,-0.988401,-0.361606,1.000000,0.999893,-0.528770,-0.064817
frepower,-0.489024,-0.987677,-0.348223,0.999893,1.000000,-0.524774,-0.063764
fpqf,0.732461,0.605058,0.428582,-0.528770,-0.524774,1.000000,0.142574
ffreq,0.418261,0.131612,0.088216,-0.064817,-0.063764,0.142574,1.000000


In [51]:
est = ols(formula = 'fpqf ~ fvoltage + fcurrent + facpower + fappower + ffreq', data = ELdata).fit()

pred = est.predict(ELdata)

In [52]:
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   fpqf   R-squared:                       0.609
Model:                            OLS   Adj. R-squared:                  0.604
Method:                 Least Squares   F-statistic:                     120.9
Date:                Thu, 05 Oct 2023   Prob (F-statistic):           7.09e-77
Time:                        08:04:10   Log-Likelihood:                 3227.2
No. Observations:                 394   AIC:                            -6442.
Df Residuals:                     388   BIC:                            -6418.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.3260      0.209      6.343      0.000       0.915       1.737
fvoltage      -0.0012      0.001     -1.347      0.179      -0.003       0.001
fcurrent      36.1148     23.377      1.545      0.123      -9.847      82.077
facpower       0.0009      0.000      3.293      0.001       0.000       0.001
fappower       0.1482      0.096      1.535      0.126      -0.042       0.338
ffreq         -0.0009      0.000     -4.592      0.000      -0.001      -0.001
==============================================================================
Omnibus:                       16.808   Durbin-Watson:                   0.278
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.541
Skew:                          -0.332   Prob(JB):                      0.00189
Kurtosis:                       2.433   Cond. No.                     1.70e+09
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 8.36e-12. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""